In [6]:
import numpy as np
import pandas as pd
import os
from collections import defaultdict
import math
import json

from tfRIM import RIMCell
from tensorflow import keras

from old_data_manip import load_data
from old_data_manip import data_aggregation
from old_data_manip import extract_train_test

In [30]:
def rmse(yhat, y):
    # print(type(yhat), type(y))
    return np.sqrt(np.mean((y - yhat)**2))


def rmspe(yhat, y):
    # EPSILON = 1e-10
    # print(((y - yhat) / (y + EPSILON))**2)
    return rmse(yhat, y) / np.mean(y)


def rmsse(yhat, y):
    e_2 = (y - yhat)
    m = 1 / (len(y) - 1)
    t = np.sum(abs(np.delete((y - np.roll(y, 1)), 0)))
    return np.sqrt(np.mean((e_2 / (m * t))**2))

In [20]:
with open('columns_selection.json') as f:
    columns_selection = json.load(f)

job_id = 6318371744

columns_to_consider = columns_selection['GWT_efficiency_1']

readings_df = load_data(f'../data/task-usage_job-ID-{job_id}_total.csv', columns_to_consider)
readings_df = data_aggregation(readings_df, aggr_type='mean')

results = defaultdict(list)

test_x, test_y, _, _, _ = extract_train_test(readings_df.values)
# print(y2)
#test_x = np.concatenate(x1, x2)
#test_y = np.concatenate(y1, y2)
#test_x =
test_x = test_x[:32,]
test_y = test_y[:32]

In [21]:
model = keras.models.load_model(
    f'../models/rim_model_exp_rim', custom_objects={'RIMCell': RIMCell}
)

yhat = model.predict(test_x)
#print(yhat.shape)

1/1 [==============================] - 1s 649ms/step


In [31]:
print(rmse(yhat, test_y)) #0.6275418
print(rmspe(yhat, test_y)) #1.3123381
print(rmsse(yhat, test_y)) #4.0115633


0.6275418
1.3123381
4.0115633
